## Housing price

https://storage.googleapis.com/ml_universities/california_housing_train.csv

что хочу:
    - импортировать в пандас, посмотреть что как, очистить, разбить на трейн и тест
    - построить TF датасет, поработать с фичами
    - построить и обучить денс-нн, учить с коллбеками, с сохранением слоев и с валидацией
    - на коллбеках сохраняться

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext tensorboard

In [ ]:
df = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep=",")

In [ ]:
df.head()

In [ ]:
# разбиваем на трейн и валидацию
df_train, df_test = train_test_split(df, train_size=0.8)

In [ ]:
df_train.describe()

In [ ]:
housing_median_age_mean = df_train['housing_median_age'].mean()
housing_median_age_std =  df_train['housing_median_age'].std()

median_income_mean = df_train['median_income'].mean()
median_income_std = df_train['median_income'].std()

# эти данные буду использовать для нормализации
housing_median_age_mean, housing_median_age_std, median_income_mean, median_income_std

In [ ]:
# то, что предсказываем
target_train = df_train.pop('median_house_value')
target_test = df_test.pop('median_house_value')

## Работа с фичами
    
    - хочу на лету создавать новые фичи - среднее число комнат на дом, среднее число жильцов на комнату
    - longitude и latitude сначала делаем бакетами, потом делаем из них крест. Т.к у нас признаком являются не сами числа, а принадлежность к квадратику
    - нормализовать все данными из датасета

In [ ]:
features = ['longitude', 'latitude', 'housing_median_age', 'population'
            'total_rooms', 'total_bedrooms','households','median_income']

In [ ]:
# прекрасно, прекрасно справляется :)
def make_new_features(x, y):
    x['family_size'] = x['population'] / x['households']
    x['pop_per_room'] = x['population'] / x['total_rooms']
    x['pop_per_bedroom'] = x['population'] / x['total_bedrooms']
    return x, y

In [ ]:
# нам это нужно и для трейна и для предикта
def create_dataset(d, t):
    ds = tf.data.Dataset.from_tensor_slices((d.to_dict('list'), t.values)).batch(4)
    ds_featured = ds.map(make_new_features)
    return ds_featured
    #layer = tf.keras.layers.DenseFeatures(features_list)
    
ds = create_dataset(df_train, target_train)
ds_train = create_dataset(df_train, target_train)
ds_test = create_dataset(df_test, target_test)

In [ ]:
for x, y in ds.take(1):
    print(x, y)

In [ ]:
# feature_column имеют немного другую роль. Они потом превращаются в слой модели,
# т.е по сути на старте они ничего и не знают про датасет, модель будет пихать в них
# данные как в мясорубку

num_features_unnorm = ['family_size', 'pop_per_room', 'pop_per_bedroom',
                       'total_rooms', 'total_bedrooms', 'households']

def create_features():
    # не важно что мы тут делаем, важно что выводим в виде итогового массива
    latitude = tf.feature_column.bucketized_column(
            tf.feature_column.numeric_column('latitude'), boundaries = np.arange(32.0, 42, 1).tolist())    
    
    longitude = tf.feature_column.bucketized_column(
            tf.feature_column.numeric_column('longitude'), boundaries = np.arange(-122, -114, 1).tolist())  
    
    squares = tf.feature_column.indicator_column(
        tf.feature_column.crossed_column([latitude, longitude], hash_bucket_size=1000))

    # нормализатор там встроен, пробую
    def norm_housing_median_age(x):
        return (x - housing_median_age_mean)/ housing_median_age_std
    housing_median_age = tf.feature_column.numeric_column('housing_median_age', 
                                                          normalizer_fn = norm_housing_median_age)

    def norm_median_income(x):
        return (x - median_income_mean)/ median_income_std
    median_income = tf.feature_column.numeric_column('median_income', 
                                                          normalizer_fn = norm_median_income)

    num_features = [tf.feature_column.numeric_column(name) for name in num_features_unnorm]
    
    return [housing_median_age, median_income, latitude] + num_features

In [ ]:
# DenseFeatures по сути просто делает тензор, на входе ему нужно подать массив "ленточек" из которых делать
# а ленточки определяются через feature_columns

layer = tf.keras.layers.DenseFeatures(create_features())

In [ ]:
for x, y in ds.take(1):
    print(layer(x).numpy())

## модель, обучение, коллбеки и вот это вот все

In [ ]:
model = tf.keras.Sequential([
    layer,
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae'])

In [ ]:
# сохранение
save_callback = tf.keras.callbacks.ModelCheckpoint('./save/model.ckpt', save_weights_only=True, verbose=1)

# тензорбоард
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir='./save/', histogram_freq=0, write_graph=True, write_images=True)



In [ ]:
history = model.fit(ds_train, epochs=10, validation_data=ds_test, callbacks=[save_callback, tensorboard])

In [ ]:
tensorboard --logdir='./save'
